In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras import Sequential
from keras import layers
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import adam_v2
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import random
from tqdm.notebook import tqdm
import collections

In [2]:
df = pd.read_csv('D:/F/data_edge/3_data/data.csv', header=None)
df = df.sort_values([1, 0])

train = np.array(df[0 : 7000].sample(7000))
train_data = train[:, 2:502] # shape (7000, 500), 100 website, 70 repeat
train_index = train[:, 0] # shape (7000,), 0-99 range, random

test = np.array(df[7000 : 10000].sort_values([0, 1]))
test_data = test[:, 2:502] # shape (3000, 500), 100 website, 70 repeat
test_index = test[:, 0] # shape (3000,), 0-99 range, sequence

In [3]:
def get_data_length(data_vector):
    data_length = 500
    for i in range(500):
        if data_vector[i] == 0:
            data_length = i
            break
    return data_length

def add_noise(data_vector, data_length, count):
    for i in range(count):
        location = random.randint(0, data_length - 1)
        packet = random.randint(-1600, 1600)
        data_vector = np.insert(data_vector, location, packet)
        if data_length < 500:
            data_length += 1
    
    return data_vector[0 : 500]

In [4]:
train_data_aug = train_data.copy()
train_index_aug = train_index.copy()
for j in range(6):
    train_data_aug_one = train_data.copy()
    for i in range(7000):
        length = get_data_length(train_data_aug_one[i])
        rate = random.randint(0, 100) / 100
        train_data_aug_one[i] = add_noise(train_data_aug_one[i], length, int(length * rate))
    train_data_aug = np.append(train_data_aug, train_data_aug_one, axis=0)
    train_index_aug = np.append(train_index_aug, train_index, axis=0)

In [5]:
model  = Sequential()
model.add(Conv1D(input_shape = (500, 1),
                 filters=32,
                 kernel_size=5,
                 padding = 'same',
                 strides=1,
                 activation='relu'))
model.add(Conv1D(filters=32,
                 kernel_size=5,
                 padding = 'same',
                 strides=1,
                 activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64,
                 kernel_size=5,
                 padding = 'same',
                 activation='relu'))
model.add(Conv1D(filters=64,
                 kernel_size=5,
                 padding = 'same',
                 activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128,
                 kernel_size=5,
                 padding = 'same',
                 activation='relu'))
model.add(Conv1D(filters=128,
                 kernel_size=5,
                 padding = 'same',
                 activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=256,
                 kernel_size=5,
                 padding = 'same',
                 activation='relu'))
model.add(Conv1D(filters=256,
                 kernel_size=5,
                 padding = 'same',
                 activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(100, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=adam_v2.Adam(learning_rate=0.001),
              metrics='accuracy')

In [6]:
model.fit(train_data_aug.astype('float32')/1600, to_categorical(train_index_aug), epochs = 10, validation_split=0.3)

Epoch 1/10
1072/1072 [==============================] - 75s 70ms/step - loss: 1.6129 - accuracy: 0.5601 - val_loss: 0.7138 - val_accuracy: 0.7872
Epoch 2/10
1072/1072 [==============================] - 114s 107ms/step - loss: 0.4798 - accuracy: 0.8527 - val_loss: 0.5326 - val_accuracy: 0.8320
Epoch 3/10
1072/1072 [==============================] - 112s 105ms/step - loss: 0.3143 - accuracy: 0.8997 - val_loss: 0.4518 - val_accuracy: 0.8628
Epoch 4/10
1072/1072 [==============================] - 135s 126ms/step - loss: 0.2436 - accuracy: 0.9208 - val_loss: 0.3983 - val_accuracy: 0.8736
Epoch 5/10
1072/1072 [==============================] - 169s 158ms/step - loss: 0.2008 - accuracy: 0.9341 - val_loss: 0.4025 - val_accuracy: 0.8821
Epoch 6/10
1072/1072 [==============================] - 171s 160ms/step - loss: 0.1597 - accuracy: 0.9466 - val_loss: 0.5138 - val_accuracy: 0.8580
Epoch 7/10
1072/1072 [==============================] - 174s 162ms/step - loss: 0.1374 - accuracy: 0.9529 - val_lo

In [7]:
model.evaluate(test_data.astype('float32')/1600, to_categorical(test_index))

94/94 [==============================] - 5s 52ms/step - loss: 0.1851 - accuracy: 0.9730


[0.1851436197757721, 0.9729999899864197]

In [8]:
model.save('data/model_tcp_aug_repeat.h5')